<a href="https://colab.research.google.com/github/anindabitm/PDFM-use-case/blob/main/PDFM_embeddings_to_predict_PM2_5_in_US.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Predicting US PM2.5 levels using Google's Population Dynamics Foundation Model**

Useful Resources:
1. https://github.com/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/zillow_home_value.ipynb
2. https://github.com/google-research/population-dynamics/tree/master/notebooks

Acknowledgements:
This notebook is based on tutorials - [PDFM notebook](https://github.com/google-research/population-dynamics/tree/master/notebooks) and awesome tutorial by giswqs opengeos PDFM [zillow home price](https://github.com/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/zillow_home_value.ipynb)

In [2]:
%%capture
!pip install leafmap

In [3]:
#import libraries
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file

# Get US PM2.5 data
Link to data: https://usc-geohealth-hub-uscssi.hub.arcgis.com/documents/7fc448343d6643f3bb13157fd65aed4f/about

In [4]:
df0 = pd.read_excel("/content/pm25_and_disparity.xlsx",sheet_name="data_part1")
df1 = pd.read_excel("/content/pm25_and_disparity.xlsx",sheet_name="data_part2")
df2 = pd.read_excel("/content/pm25_and_disparity.xlsx",sheet_name="data_part3")
df3 = pd.read_excel("/content/pm25_and_disparity.xlsx",sheet_name="data_part4")
df4 = pd.read_excel("/content/pm25_and_disparity.xlsx",sheet_name="data_part5")

# Process PM2.5 data

In [5]:
df = pd.concat([df0,df1,df2,df3,df4],ignore_index=True)
df.head()

,Unnamed: 0,year,ZIP,zcta,popdensity,population,poverty,education,pct_blk,pct_hisp,pct_native,pct_asian,pct_white,black_pop,hisp_pop,native_pop,asian_pop,white_pop,medhouseholdincome,pm25,urban
0,1,2000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2002,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2003,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.shape

(789260, 21)

In [7]:
df["zcta"].nunique()

32406

In [8]:
pm25_df = df.groupby(["zcta"]).mean()["pm25"]
pm25_df.head()

,pm25
zcta,
601.0,NaN
602.0,NaN
603.0,NaN
606.0,NaN
610.0,NaN


In [9]:
pm25_df.dropna(axis=0,inplace=True)
pm25_df.head()

,pm25
zcta,
1001.0,9.398180
1002.0,8.026795
1003.0,8.949020
1005.0,6.409811
1007.0,7.375929


In [10]:
pm25_df.index = pm25_df.index.astype(int)
print(pm25_df.shape)
pm25_df.head()

(31956,)


,pm25
zcta,
1001,9.398180
1002,8.026795
1003,8.949020
1005,6.409811
1007,7.375929


In [11]:
pm25_df = pm25_df.reset_index(drop=False)  # Remove inplace=True
pm25_df.index = pm25_df["zcta"].apply(lambda x: f"zip/{x}") # Access 'zcta' column
pm25_df.head()

,zcta,pm25
zcta,,
zip/1001,1001,9.398180
zip/1002,1002,8.026795
zip/1003,1003,8.949020
zip/1005,1005,6.409811
zip/1007,1007,7.375929


# Request access to PDFM Embeddings

In [14]:
!unzip /content/pdfm_embeddings.zip

Archive:  /content/pdfm_embeddings.zip
   creating: pdfm_embeddings/
   creating: pdfm_embeddings/v0/
   creating: pdfm_embeddings/v0/us/
  inflating: pdfm_embeddings/v0/us/county.geojson  
  inflating: pdfm_embeddings/v0/us/county_embeddings.csv  
  inflating: pdfm_embeddings/v0/us/zcta.geojson  
  inflating: pdfm_embeddings/v0/us/zcta_embeddings.csv  


In [12]:
embeddings_file_path = "/content/pdfm_embeddings/v0/us/zcta_embeddings.csv"

In [15]:
if not os.path.exists(embeddings_file_path):
    raise FileNotFoundError("Please request the embeddings from Google")

In [16]:
zipcode_embeddings = pd.read_csv(embeddings_file_path).set_index("place")
zipcode_embeddings.head()

,state,county,city,population,latitude,longitude,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,feature42,feature43,...,feature280,feature281,feature282,feature283,feature284,feature285,feature286,feature287,feature288,feature289,feature290,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300,feature301,feature302,feature303,feature304,feature305,feature306,feature307,feature308,feature309,feature310,feature311,feature312,feature313,feature314,feature315,feature316,feature317,feature318,feature319,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
place,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
zip/97910,OR,Malheur County,Jordan Valley,609,42.749076,-117.511459,-0.138227,1.120377,0.072900,0.297442,0.772673,0.286467,0.802398,0.620847,0.060810,0.125926,0.452905,0.317210,1.560488,0.173717,0.338584,-0.011876,0.369918,0.734241,-0.023161,0.927918,0.131129,0.174915,0.186962,0.777327,-0.125255,0.252997,0.126703,0.282713,0.286217,0.108222,0.138043,1.893893,0.034341,0.116197,1.578618,0.594598,0.083396,0.217705,0.085833,0.050959,0.071389,0.046794,0.900741,0.089737,...,0.795463,4.048826,4.411071,-0.072585,1.172804,2.780721,-0.007037,-0.167074,-0.169071,2.451642,4.705745,2.861044,0.964761,5.425952,-0.086446,-0.004044,1.692680,-0.129304,1.262767,0.584393,4.957980,-0.113619,3.977844,-0.056266,0.154679,6.833614,-0.168595,3.852105,-0.008600,0.168367,2.672679,6.938071,1.462526,4.700379,3.523755,-0.169971,0.279797,-0.030630,-0.000014,4.489360,-0.158891,-0.168708,1.231994,-0.155765,3.043214,-0.169749,0.177463,-0.001661,-0.001010,4.495589
zip/89412,NV,Washoe County,Gerlach,98,41.102934,-119.695361,-0.141379,1.422782,0.234269,0.159156,0.890241,0.215427,0.533200,1.125830,0.159891,0.305449,0.673448,0.222371,1.113196,0.147047,0.270858,0.140051,0.591381,1.321858,0.182956,1.600126,0.149480,1.239454,0.326186,0.724274,-0.057049,0.327722,0.225044,0.092865,0.636425,0.541034,0.168866,1.489893,-0.003255,0.524351,1.672642,0.421262,0.701539,0.231094,0.223512,0.134480,0.479852,0.204405,1.043766,0.396896,...,-0.080731,5.357723,3.973490,-0.100555,2.942436,4.120401,-0.027611,-0.084821,-0.000068,1.364740,6.085680,5.550473,-0.057929,7.677364,-0.111488,0.442242,2.686013,-0.008651,-0.003287,4.685384,6.117020,-0.083324,4.441121,-0.002397,-0.119943,4.539424,-0.006676,4.993636,-0.118508,-0.169039,1.864953,4.146715,-0.118220,3.688882,4.046134,-0.045537,1.627209,-0.012242,-0.016643,4.668972,-0.157417,-0.043606,2.788701,-0.062547,3.700745,-0.169827,-0.137990,-0.024385,-0.000295,3.399393
zip/88030,NM,Luna County,Deming,24139,32.191634,-107.729431,-0.046666,1.414424,0.146803,1.113256,1.119576,1.093199,0.960636,0.179642,0.729488,2.447439,2.274204,2.765325,0.903284,0.520162,2.604348,0.688520,0.164436,2.755828,1.312246,0.452269,0.612660,1.135295,1.440466,0.507069,-0.140809,0.481306,1.068717,2.188697,0.254398,1.019234,0.277727,0.716491,0.861136,1.232256,0.210326,0.694031,1.504018,1.430361,0.842551,0.222043,1.114556,0.856425,1.518791,1.487212,...,0.472449,4.089555,1.347347,0.128349,2.517892,0.718034,0.546053,2.494347,-0.024888,3.658147,5.096304,3.687950,-0.162521,4.844636,0.789083,3.025670,0.667166,-0.169164,-0.109891,3.095128,4.816823,-0.169883,4.487709,-0.058803,-0.167333,2.843048,-0.060544,3.279263,-0.158699,1.535189,2.787231,3.861916,1.569119,3.487299,2.334693,0.068785,-0.162307,-0.053849,0.160504,1.895565,-0.000654,0.437475,4.229295,0.229199,2.098469,1.150497,0.716122,-0.116499,-0.051163,3.866543
zip/82633,WY,Converse County,Douglas,9478,43.022270,-

# Join PDFM embeddings and Groud Truth (PM2.5 data)

In [17]:
data = pm25_df.join(zipcode_embeddings, how="inner")
data.head()

,zcta,pm25,state,county,city,population,latitude,longitude,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,...,feature280,feature281,feature282,feature283,feature284,feature285,feature286,feature287,feature288,feature289,feature290,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300,feature301,feature302,feature303,feature304,feature305,feature306,feature307,feature308,feature309,feature310,feature311,feature312,feature313,feature314,feature315,feature316,feature317,feature318,feature319,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
zip/10001,10001,13.636975,NY,New York County,New York,26966,40.750672,-73.997281,-0.073025,0.621611,3.204059,1.219250,0.086666,0.422733,1.362252,0.448227,0.780789,0.076176,0.744889,0.206153,1.486222,2.341626,2.934966,3.197650,2.227243,0.536704,1.641904,0.215733,0.084727,0.164679,4.511102,2.252191,-0.024315,0.212265,3.857031,2.217771,0.308766,4.034735,1.695519,0.567224,-0.077346,3.953824,1.425099,2.275840,1.042735,1.490175,1.557218,0.206169,0.049573,2.313412,...,1.501178,-0.123201,-0.167260,-0.159472,-0.001858,-0.129980,0.175728,-1.200000e-07,-0.008414,-0.155363,0.315214,-0.009080,-0.0,-0.000000e+00,0.770695,-0.152233,-0.049659,8.209490,0.700820,1.000827,5.112181,-0.146953,1.654920,-0.002328,-0.004713,-0.162020,9.814237,2.872081,7.905857,9.632621,7.280638,-0.017287,0.479747,1.191547,6.030477,-0.028483,-0.002579,-0.150458,-0.145124,-0.000732,-0.006855,2.571859,-0.006076,4.313338,-0.105290,-9.500000e-06,1.581655,-0.015907,-0.0,3.977647
zip/10002,10002,12.896079,NY,New York County,New York,76807,40.715762,-73.986258,-0.144769,1.102272,3.980774,1.751827,0.082982,0.474515,0.949866,0.379872,1.394939,0.239189,0.831812,0.231631,1.985357,2.112055,3.209443,3.651924,1.369114,0.449409,2.278484,0.187911,0.342743,0.178030,5.858740,2.435344,-0.011125,0.203172,3.586207,2.016004,0.292903,4.168702,2.146975,1.026136,-0.085150,4.931133,1.664730,2.572370,0.716705,2.225487,2.063185,0.276354,0.272357,3.370650,...,0.361146,-0.096761,-0.146408,-0.155304,-0.074113,-0.015054,1.665584,-1.102000e-05,-0.002935,-0.169603,0.145326,-0.025443,-0.0,-5.000000e-08,-0.161510,-0.153942,-0.166410,6.376185,1.163428,1.138928,5.270078,-0.159924,1.996964,-0.000041,-0.020128,-0.050640,8.771538,2.086100,8.602596,8.060193,8.714463,-0.004688,0.136199,0.513982,5.643821,-0.002350,-0.008118,-0.089103,-0.156498,-0.034799,-0.103022,2.710451,-0.000021,4.655519,-0.159014,-0.000000e+00,5.388407,-0.112462,-0.0,4.431747
zip/10003,10003,12.829412,NY,New York County,New York,54447,40.731829,-73.989181,-0.131461,1.132039,3.512190,1.176800,0.161716,0.459370,1.291074,0.475496,1.146470,0.199002,0.968685,0.249058,1.788139,2.295915,3.039128,3.726780,1.822461,0.546915,1.955472,0.188880,0.198170,0.138593,5.482426,2.644991,-0.030993,0.256750,3.747168,2.160955,0.311789,4.131626,2.050255,0.924002,-0.077789,4.401683,1.490332,2.379128,0.871206,2.151970,1.929199,0.361298,0.229650,3.038753,...,0.461322,-0.141985,-0.085863,-0.156253,-0.005502,-0.046846,1.596130,-4.810000e-06,-0.089342,-0.099540,0.272569,-0.009312,-0.0,-0.000000e+00,0.092946,-0.169363,-0.042557,6.376621,0.606484,0.778049,5.227136,-0.169888,2.206395,-0.000619,-0.166381,-0.167345,8.366828,2.290446,7.762557,7.947579,6.410265,-0.020947,0.935508,0.415192,5.355077,-0.027503,-0.000485,-0.123136,0.030198,-0.001412,-0.004739,2.824203,-0.000298,4.061482,-0.068253,-1.350000e-06,4.039156,-0.156848,-0.0,5.094444
zip/10004,10004,13.851765,NY,New York County,New York,4795,40.688630,-74.018244,-0.147625,0.546787,3.229492,1.886864,0.108621,0.603010,1

In [18]:
data.shape

(29298, 338)

In [19]:
embedding_features = [f"feature{x}" for x in range(330)]
label = "pm25"

In [20]:
data = data.dropna(subset=[label])

# Split Train and Test Data

In [21]:

data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit K-Nearest Neighbors Model

In [22]:
k = 5
model = KNeighborsRegressor(n_neighbors=k)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [23]:

evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
# Evaluate the model
metrics = evaluate_model(evaluation_df)
print(metrics)

{'r2': 0.9423382724431623, 'r': 0.9708871328512507, 'rmse': 0.6494730499555952, 'mae': 0.4204252179218217, 'mape': 0.05619889943563576}


# Evaluate K-Nearest Neighbors Model

In [28]:
xy_lim = (0, 30)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted PM2.5",
    x_label="Actual PM2.5",
    y_label="Predicted PM2.5",
)

# Fit Random Forest Regressor model

In [29]:
model = RandomForestRegressor(n_estimators=10,verbose=10,n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 10building tree 2 of 10



[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   20.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10


[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   38.8s


building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished


In [30]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
# Evaluate the model
metrics = evaluate_model(evaluation_df)
print(metrics)

{'r2': 0.9086928793143653, 'r': 0.9539294211355118, 'rmse': 0.817277670967286, 'mae': 0.568816793219495, 'mape': 0.07572915522396176}


# Evaluate Random Forest Model

In [31]:
xy_lim = (0, 30)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted PM2.5",
    x_label="Actual PM2.5",
    y_label="Predicted PM2.5",
)